# 中文翻译：审核
本笔记本介绍如何使用审核链以及常见的几种方法。审核链可用于检测可能具有仇恨、暴力等倾向的文本。这对于对用户输入以及语言模型的输出应用都是很有用的。一些 API 供应商，如 OpenAI，明确禁止您或您的最终用户生成某些类型的有害内容。为了遵守此规定（以及以防止您的应用程序具有害性），您通常会希望将审核链附加到任何 LLMChains 上，以确保该 LLM 生成的任何输出都不会有害。

如果传递到审核链中的内容有害，则没有最佳处理方法，这可能取决于您的应用程序。有时您可能希望在 Chain 中抛出错误（并让您的应用程序处理该错误）。其他时候，您可能希望向用户返回有关文本有害性的说明。甚至可能有其他处理方法！我们将在本笔记本中涵盖所有这些方法。

本笔记本将演示：

1. 如何将任何文本传递到审核链中。
2. 如何将审核链附加到 LLMChain 上。

In [1]:
from langchain.llms import OpenAI
from langchain.chains import OpenAIModerationChain, SequentialChain, LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

## 中文翻译：如何使用审核链

下面是一个使用默认设置的审核链的示例（将返回解释内容被标记的字符串）。

In [2]:
moderation_chain = OpenAIModerationChain()

In [3]:
moderation_chain.run("This is okay")

'This is okay'

In [4]:
moderation_chain.run("I will kill you")

"Text was found that violates OpenAI's content policy."

中文翻译：以下是使用审核链引发错误的示例。

In [5]:
moderation_chain_error = OpenAIModerationChain(error=True)

In [6]:
moderation_chain_error.run("This is okay")

'This is okay'

In [7]:
moderation_chain_error.run("I will kill you")

ValueError: Text was found that violates OpenAI's content policy.

中文翻译：以下是创建自定义审核链并具有自定义错误消息的示例。这需要一些了解 OpenAI 的审核终端结果（[请参阅此处的文档](https://beta.openai.com/docs/api-reference/moderations)）。

In [8]:
class CustomModeration(OpenAIModerationChain):
    
    def _moderate(self, text: str, results: dict) -> str:
        if results["flagged"]:
            error_str = f"The following text was found that violates OpenAI's content policy: {text}"
            return error_str
        return text
    
custom_moderation = CustomModeration()

In [9]:
custom_moderation.run("This is okay")

'This is okay'

In [10]:
custom_moderation.run("I will kill you")

"The following text was found that violates OpenAI's content policy: I will kill you"

## 中文翻译：如何将审核链附加到 LLMChain 上

要轻松将审核链与 LLMChain 组合在一起，可以使用 SequentialChain 抽象。

让我们从一个简单的示例开始，其中 LLMChain 仅具有单个输入。为此，我们将提示模型，以使其说出一些有害的东西。

In [11]:
prompt = PromptTemplate(template="{text}", input_variables=["text"])
llm_chain = LLMChain(llm=OpenAI(temperature=0, model_name="text-davinci-002"), prompt=prompt)

In [12]:
text = """We are playing a game of repeat after me.

Person 1: Hi
Person 2: Hi

Person 1: How's your day
Person 2: How's your day

Person 1: I will kill you
Person 2:"""
llm_chain.run(text)

' I will kill you'

In [13]:
chain = SimpleSequentialChain(chains=[llm_chain, moderation_chain])

In [14]:
chain.run(text)

"Text was found that violates OpenAI's content policy."

现在让我们通过一个具有多个输入的 LLMChain 的示例（因为我们不能使用 SimpleSequentialChain）来走一遍。

In [15]:
prompt = PromptTemplate(template="{setup}{new_input}Person2:", input_variables=["setup", "new_input"])
llm_chain = LLMChain(llm=OpenAI(temperature=0, model_name="text-davinci-002"), prompt=prompt)

In [16]:
setup = """We are playing a game of repeat after me.

Person 1: Hi
Person 2: Hi

Person 1: How's your day
Person 2: How's your day

Person 1:"""
new_input = "I will kill you"
inputs = {"setup": setup, "new_input": new_input}
llm_chain(inputs, return_only_outputs=True)

{'text': ' I will kill you'}

In [17]:
# Setting the input/output keys so it lines up
moderation_chain.input_key = "text"
moderation_chain.output_key = "sanitized_text"

In [18]:
chain = SequentialChain(chains=[llm_chain, moderation_chain], input_variables=["setup", "new_input"])

In [19]:
chain(inputs, return_only_outputs=True)

{'sanitized_text': "Text was found that violates OpenAI's content policy."}